## Fonctions necessaire 

In [2]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += images.size(0)

    return total_loss / total, correct / total

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += images.size(0)

    return total_loss / total, correct / total

def train_model(model, train_loader, val_loader, num_epochs=20, save_dir="DenseNet121"):
    os.makedirs(save_dir, exist_ok=True)
    device_ = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device_)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    best_val_acc = 0.0
    best_model_wts = model.state_dict()
    log_file_path = os.path.join(save_dir, "training_log.txt")

    with open(log_file_path, "w") as log_file:
        for epoch in range(1, num_epochs + 1):
            train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device_)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device_)

            log = (f"Epoch {epoch:02d} | TRAIN loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
                   f"VALID loss: {val_loss:.4f}, acc: {val_acc:.4f} | LR: {optimizer.param_groups[0]['lr']:.2e}")
            print(log)
            all_losses_train.append(train_loss)
            all_losses_val.append(val_loss)
            all_acc_train.append(train_acc)
            all_acc_val.append(val_acc)

            log_file.write(log + "\n")

            scheduler.step(val_loss)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_wts = model.state_dict()

    save_path = os.path.join(save_dir, f"best_densenet121_acc{best_val_acc:.4f}.pth")
    torch.save(best_model_wts, save_path)
    print(f" Meilleur modèle sauvegardé : {save_path}")
    return model


# Ici on funetune sur notre nouveau jeu de données 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import os


In [2]:
data_dir = "captured_faces_finetunning"  # ton nouveau dossier

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
class_names = dataset.classes
num_classes = len(class_names)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Créer le modèle avec 4 classes (comme celui sauvegardé)
model = models.densenet121(pretrained=False)
model.classifier = nn.Linear(model.classifier.in_features, 4)
model.load_state_dict(torch.load("DenseNet121/best_densenet121_acc0.7663.pth"))

model = model.to(device)



/home/grp1issd/Desktop/Projet_TPE/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/grp1issd/Desktop/Projet_TPE/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

best_acc = 0

for epoch in range(10):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += inputs.size(0)

    val_correct, val_total = 0, 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total * 100
    print(f"Epoch {epoch+1} - Loss: {running_loss/total:.4f}, Val Acc: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "finetuned_model/fine_tuned_model.pth")
        print(" Nouveau meilleur modèle sauvegardé.")


Epoch 1 - Loss: 0.5613, Val Acc: 88.64%
 Nouveau meilleur modèle sauvegardé.
Epoch 2 - Loss: 0.1403, Val Acc: 95.45%
 Nouveau meilleur modèle sauvegardé.
Epoch 3 - Loss: 0.0634, Val Acc: 93.18%
Epoch 4 - Loss: 0.0684, Val Acc: 95.45%
Epoch 5 - Loss: 0.0611, Val Acc: 100.00%
 Nouveau meilleur modèle sauvegardé.
Epoch 6 - Loss: 0.0536, Val Acc: 95.45%
Epoch 7 - Loss: 0.0162, Val Acc: 95.45%
Epoch 8 - Loss: 0.0231, Val Acc: 97.73%
Epoch 9 - Loss: 0.0312, Val Acc: 95.45%
Epoch 10 - Loss: 0.0367, Val Acc: 95.45%
